In [ ]:
import numpy as np  # Load required libs
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from tqdm.notebook import tqdm
from torch.nn import ReLU, Conv2d, BatchNorm2d, Sequential, AdaptiveAvgPool2d, Linear, MaxPool2d
from torch.nn.functional import one_hot
from typing import Tuple, List, Type, Dict, Any
from threading import Thread, Lock
from queue import Empty, Queue
from sklearn.utils import shuffle
from torch.autograd import Variable
import cv2
import os
import pickle
import datetime
import zipfile
import copy
import time
import sys
try:
    import adabelief_pytorch
except:
    !pip install adabelief_pytorch==0.2.0
    time.sleep(1)
    import adabelief_pytorch
import gc  # sometimes it is required to clean GPU memory in that way

In [ ]:
import sys
sys.path.append('libs')

from dataset import CelebADataset, FlexDataloader
from threading_data import Thread_data_processing
from train_val import train_model
from network_collection import UnitedModel

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # We'd like to use GPU
device

device(type='cuda', index=0)

In [ ]:
test_transform = torchvision.transforms.Compose([ToFloat(),
                                                 torchvision.transforms.Normalize(mean=(129.0484, 108.4658,  97.5840), std=(79.0141, 73.8808, 73.7167))])
train_transform = torchvision.transforms.Compose([torchvision.transforms.AutoAugment(torchvision.transforms.AutoAugmentPolicy.IMAGENET),
                                                  ToFloat(),
                                                  torchvision.transforms.Normalize(mean=(129.0484, 108.4658,  97.5840), std=(79.0141, 73.8808, 73.7167))])

In [ ]:
train_dataset, test_dataset = CelebADataset.make_train_test_datasets(extract_path = 'data', 
                                                                     zip_path="img_align_celeba.zip",
                                                                     annotation_path = "identity_CelebA.txt")

In [ ]:
train_dataloader = FlexDataloader(batch_size=16, train_dataset)
test_dataloader = FlexDataloader(batch_size=16, test_dataset)

In [ ]:
len(train_dataloader)

50000

In [ ]:
len(test_dataloader)

10000

In [ ]:
model = UnitedModel(self,
                 out_features_classifier=8369,
                 scale=64,
                 margins=(1.2, 0.05, 0.004),
                 embedding_dim=128,
                 cut_fc=False,
                 reference_resnet='50',
                 after_relu=False)
print('Total number of trainable parameters : {}'.format(sum(parameter.numel() for parameter in model.parameters() if parameter.requires_grad)))

Total number of trainable parameters : 363552


In [ ]:
datagiver = Thread_data_processing(train_generator = train_gen, 
                                   val_generator = val_gen,
                                   device = device, 
                                   img_transform_train = train_transform, 
                                   img_transform_val = test_transform,
                                   ram_queue_length=12,
                                   cuda_queue_length=5,
                                   num_workers_ram=3,
                                   num_workers_cuda=1)
datagiver.start()

process has been started
process has been started
process has been started
process has been started
process has been started
process has been started


In [ ]:
adabelief_params = {
            'lr' : 5e-4,
            'betas' : (0.9, 0.999),
            'eps' : 1e-8,
            'weight_decay' : 1e-2,
            'weight_decouple' : True,
            'rectify' : False,
            'fixed_decay' : False,
            'amsgrad' : False
        }

cosine_anneling = {
            'eta_min' : 1e-5,
            'T_0' : 1
        }


train_model(model = model, 
            datagiver = datagiver,
            path_to_save = 'best_models',
            optimizer_params = adabelief_params,
            lr_scheduler_params = cosine_anneling,
            lr_scheduler_step_every_epoch = True,
            max_epochs = 5,
            early_stopping_patience = 4,
            loss_progression = None)

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Number of hot threads - 6
threads has been frozen
threads has been frozen
threads has been frozen
threads has been frozen
threads has been frozen
Number of hot threads - 1
thread has been frozen
Number of hot threads - 0
process has been started
process has been started
process has been started
process has been started
process has been started
process has been started
6 processes have been starter


  0%|          | 0/782 [00:00<?, ?it/s]

Number of hot threads - 6threads has been frozen

threads has been frozen
threads has been frozen
threads has been frozen
threads has been frozen
thread has been frozen
Number of hot threads - 0
process has been started
process has been started
process has been started
3 processes have been starter
process has been started
process has been started
process has been started
correct - 0.821656050955414 , loss - 0.5211427875195339
Best model yet, saving


FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/GitHub/NN studying/NN_studying/metric learning - face recognition(like FaceID)/best_models/Classificator_best.tp'